# Récupération des données pour l'étude départementale

## 1- Importation des librairies externes et classiques

In [1]:
# Importation des librairies externes
!pip install git+https://github.com/InseeFrLab/Py-Insee-Data@master --ignore-install PyYAML
!pip install python-Levenshtein
!pip install openpyxl

  Cloning https://github.com/InseeFrLab/Py-Insee-Data (to revision master) to /tmp/pip-req-build-ti2s8ytr
  Running command git clone --filter=blob:none -q https://github.com/InseeFrLab/Py-Insee-Data /tmp/pip-req-build-ti2s8ytr
  Resolved https://github.com/InseeFrLab/Py-Insee-Data to commit 52139da402bc33da6b8892d7d3208f1d2ddfaa02
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     |████████████████████████████████| 661 kB 1.7 MB/s            
     |████████████████████████████████| 235 kB 49.2 MB/s            
     |████████████████████████████████| 76 kB 13.9 MB/s            
     |████████████████████████████████| 60 kB 19.3 MB/s            
     |████████████████████████████████| 11.5 MB 41.1 MB/s            
     |████████████████████████████████| 62 kB 2.0 MB/s             
     |████████████████████████████████| 255 kB 22.2 MB/s            
     |████████████████████████████████| 503 kB 25.6

In [2]:
# Importation des librairies 
import requests
import pandas as pd
import pynsee
import pynsee.download
from pynsee.sirene import *
import time
from tqdm import tqdm
from pynsee.utils.init_conn import init_conn
import os
from zipfile import ZipFile, Path
import io

In [3]:
# token API insee
init_conn(insee_key=os.environ['INSEE_KEY'], insee_secret=os.environ['INSEE_SECRET'])

## 2- Téléchargement des données de SIRENE

In [4]:
### Téléchargement des données Sirene sur les établissements Siret de boucherie-charcuterie (code 4722Z) à l'instant T
# Téléchargement via Insee sirene data  by sirene.fr website

session = requests.session()
URL_ACCUEIL = "https://www.sirene.fr/sirene/public/creation-fichier"
URL_REQUETE_POST = "https://www.sirene.fr/sirene/public/verifier-creation-fichier.action"
URL_DEVIS="https://www.sirene.fr/sirene/client//sirene/client/devis-demande.action"

URL_RECAP="https://www.sirene.fr/sirene/client/telechargement-pret-fichier.action"
URL_TELECHARGEMENT_POST="https://www.sirene.fr/sirene/client/telechargement-fichier"
POST_DATA={
'fichier.caracteristiques.etat.actif': 'true',
'__checkbox_fichier.caracteristiques.etat.actif': 'true',
'__checkbox_fichier.caracteristiques.etat.ferme': 'true',
'fichier.caracteristiques.typeEtablissement': '1',
'fichier.caracteristiques.geoFile': 'false',
'fichier.caracteristiques.cjFile': 'false',
'fichier.caracteristiques.apeFile': 'false',
'fichier.caracteristiques.sirenSiretFile': 'false',
'fichiersCriteres.criteresGeographie.upload': '(binaire)',
'__multiselect_fichier.criteres.geographie.region.modalites': '',
'__multiselect_fichier.criteres.geographie.departement.modalites': '',
'fichiersCriteres.criteresActivite.upload': '(binaire)',
'__multiselect_fichier.criteres.activite.section.modalites': '',
'__multiselect_fichier.criteres.activite.division.modalites': '',
'__multiselect_fichier.criteres.activite.groupe.modalites': '',
'fichier.criteres.activite.sousClasse.modalites': '4722Z',
'__multiselect_fichier.criteres.activite.sousClasse.modalites': '',
'fichiersCriteres.criteresNature.upload': '(binaire)',
'__multiselect_fichier.criteres.nature.entrepriseNiveau1.modalites': '',
'__multiselect_fichier.criteres.nature.entrepriseNiveau2.modalites': '',
'__multiselect_fichier.criteres.nature.entrepriseNiveau3.modalites': '',
'__multiselect_fichier.criteres.trancheEffectif.modalites': '',
'fichier.criteres.dateCreation.date': '',
'fichier.criteres.dateCreation.dateFin': '',
'fichier.criteres.dateMaj.date': '',
'fichier.criteres.dateMaj.dateFin': '',
'fichiersCriteres.criteresSirenSiret.upload': '(binaire)',
'__multiselect_fichier.criteres.criteresCatEnt.modalites': '',
'strutsLocale': 'fr_FR',
}

result=session.get(URL_ACCUEIL)
result=session.post(URL_REQUETE_POST, data=POST_DATA)


result=session.get(URL_RECAP)

result=session.get(URL_DEVIS)

result=session.post(URL_TELECHARGEMENT_POST)

donnees_sirene_fr=pd.read_csv(io.StringIO(result.content.decode('utf-8')),dtype=str)


In [5]:
donnees_sirene_fr.shape
donnees_sirene_fr.sample(10)

,siren,nic,siret,statutDiffusionEtablissement,dateCreationEtablissement,trancheEffectifsEtablissement,anneeEffectifsEtablissement,activitePrincipaleRegistreMetiersEtablissement,dateDernierTraitementEtablissement,etablissementSiege,...,codePaysEtranger2Etablissement,libellePaysEtranger2Etablissement,etatAdministratifEtablissement,enseigne1Etablissement,enseigne2Etablissement,enseigne3Etablissement,denominationUsuelleEtablissement,activitePrincipaleEtablissement,nomenclatureActivitePrincipaleEtablissement,caractereEmployeurEtablissement
14395,819973579,00017,81997357900017,O,2016-05-02,01,2019,4722ZB,2021-10-27T09:25:36,true,...,NaN,NaN,A,AN NOUR,NaN,NaN,NaN,47.22Z,NAFRev2,O
5981,452136575,00035,45213657500035,O,2021-05-21,NaN,NaN,4722ZB,2021-06-21T10:02:01,true,...,NaN,NaN,A,NaN,NaN,NaN,LA DUSSACOISE,47.22Z,NAFRev2,N
9457,522728732,00010,52272873200010,O,2010-05-26,02,2019,4722ZB,2021-10-27T08:30:16,true,...,NaN,NaN,A,BOUCHERIE DE L'HERBAGE,NaN,NaN,NaN,47.22Z,NAFRev2,O
5172,440862449,00012,44086244900012,O,2002-01-31,02,2019,4722ZB,2021-10-27T08:20:53,true,...,NaN,NaN,A,NaN,NaN,NaN,NaN,47.22Z,NAFRev2,O
3008,400986147,00017,40098614700017,O,1995-05-15,NN,NaN,4722ZB,2015-03-17T00:44:18,true,...,NaN,NaN,A,NaN,NaN,NaN,NaN,47.22Z,NAFRev2,O
849,330328782,00017,33032878200017,O,1984-07-01,03,2019,4722ZB,2021-10-27T08:27:50,true,...,NaN,NaN,A,NaN,NaN,NaN,NaN,47.22Z,NAFRev2,O
2955,400432944,00017,40043294400017,O,1995-03-17,01,2019,4722ZA,2021-10-27T08:26:50,true,...,NaN,NaN,A,NaN,NaN,NaN,NaN,47.22Z,NAFRev2,O
3326,407995695,00019,40799569500019,O,1996-06-01,NN,NaN,NaN,2008-12-06T05:15:51,true,...,NaN,NaN,A,NaN,NaN,NaN,NaN,47.22Z,NAFRev2,N
1297,343079448,00050,34307944800050,O,2010-01-01,NN,NaN,4722ZA,2018-05-22T09:01:35,true,...,NaN,NaN,A,NaN,NaN,NaN,NaN,47.22Z,NAFRev2,O
4283,422801498,00029,42280149800029,O,2007-01-01,00,2018,4722ZA,2021-02-23T11:44:32,true,...,NaN,NaN,A,BOUCHERIE DU MAGHREB,NaN,NaN,NaN,47.22Z,NAFRev2,N


* les données de Sirene.fr sont des données d'entreprises France entière. Pour l'étude, le nombre d'établissement par département est considéré :

In [22]:
pd.set_option('display.max_columns', None)
donnees_sirene_fr.sample(10)

,siren,nic,siret,statutDiffusionEtablissement,dateCreationEtablissement,trancheEffectifsEtablissement,anneeEffectifsEtablissement,activitePrincipaleRegistreMetiersEtablissement,dateDernierTraitementEtablissement,etablissementSiege,etatAdministratifUniteLegale,statutDiffusionUniteLegale,unitePurgeeUniteLegale,dateCreationUniteLegale,categorieJuridiqueUniteLegale,denominationUniteLegale,sigleUniteLegale,denominationUsuelle1UniteLegale,denominationUsuelle2UniteLegale,denominationUsuelle3UniteLegale,sexeUniteLegale,nomUniteLegale,nomUsageUniteLegale,prenom1UniteLegale,prenom2UniteLegale,prenom3UniteLegale,prenom4UniteLegale,prenomUsuelUniteLegale,pseudonymeUniteLegale,activitePrincipaleUniteLegale,nomenclatureActivitePrincipaleUniteLegale,identifiantAssociationUniteLegale,economieSocialeSolidaireUniteLegale,caractereEmployeurUniteLegale,trancheEffectifsUniteLegale,anneeEffectifsUniteLegale,nicSiegeUniteLegale,dateDernierTraitementUniteLegale,categorieEntreprise,anneeCategorieEntreprise,complementAdresseEtablissement,numeroVoieEtablissement,indiceRepetitionEtablissement,typeVoieEtablissement,libelleVoieEtablissement,codePostalEtablissement,libelleCommuneEtablissement,libelleCommuneEtrangerEtablissement,distributionSpecialeEtablissement,codeCommuneEtablissement,codeCedexEtablissement,libelleCedexEtablissement,codePaysEtrangerEtablissement,libellePaysEtrangerEtablissement,complementAdresse2Etablissement,numeroVoie2Etablissement,indiceRepetition2Etablissement,typeVoie2Etablissement,libelleVoie2Etablissement,codePostal2Etablissement,libelleCommune2Etablissement,libelleCommuneEtranger2Etablissement,distributionSpeciale2Etablissement,codeCommune2Etablissement,codeCedex2Etablissement,libelleCedex2Etablissement,codePaysEtranger2Etablissement,libellePaysEtranger2Etablissement,etatAdministratifEtablissement,enseigne1Etablissement,enseigne2Etablissement,enseigne3Etablissement,denominationUsuelleEtablissement,activitePrincipaleEtablissement,nomenclatureActivitePrincipaleEtablissement,caractereEmployeurEtablissement
12334,799858634,00030,79985863400030,O,2018-04-01,NaN,NaN,4722ZB,2019-11-14T14:00:35,false,A,O,NaN,2014-01-20,5499,MORICE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.22Z,NAFRev2,NaN,NaN,O,03,2019,00014,2021-10-27T09:06:53,PME,2019,NaN,21,NaN,RUE,NATIONALE,37160,LA CELLE-SAINT-AVANT,NaN,NaN,37045,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,BOUCHERIE CHARCUTERIE ALIMENTATION,NaN,NaN,BOUCHERIE CHARCUTERIE ALIMENTATION MORICE,47.22Z,NAFRev2,O
11765,792476947,00017,79247694700017,O,2013-03-15,NaN,NaN,4722ZB,2020-02-04T08:46:56,true,A,O,NaN,2013-03-15,1000,NaN,NaN,NaN,NaN,NaN,M,GIRAULT,NaN,FLORIAN,JEAN-YVES,ROSELIN,NaN,FLORIAN,NaN,47.22Z,NAFRev2,NaN,NaN,N,NaN,NaN,00017,2017-05-20T02:43:19,PME,2019,NaN,57,NaN,RUE,GAMBETTA,50100,CHERBOURG-EN-COTENTIN,NaN,NaN,50129,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,BOUCHERIE NORMANDE,NaN,NaN,NaN,47.22Z,NAFRev2,N
3197,403222524,00015,40322252400015,O,1995-12-21,02,2019,4722ZB,2021-10-27T08:29:30,true,A,O,NaN,1995-12-21,5499,BOUCHERIE CHARCUTERIE DE L'EYRIEUX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.22Z,NAFRev2,NaN,N,O,11,2019,00015,2021-10-27T08:29:30,PME,2019,NaN,NaN,NaN,RUE,DES ECOLES,07190,SAINT-SAUVEUR-DE-MONTAGUT,NaN,NaN,07295,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,NaN,NaN,NaN,NaN,47.22Z,NAFRev2,O
8605,509489225,00029,50948922500029,O,2018-06-19,NaN,NaN,4722ZB,2019-11-14T14:00:15,false,A,O,NaN,2008-12-01,5499,BOUCHERIE CHARCUTERIE MOULIN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.22Z,NAFRev2,NaN,NaN,O,02,2019,00011,2021-10-27T08:14:21,PME,2019,NaN,9,NaN,AV,PAUL RIBEYRE,07600,VALS-LES-BAINS,NaN,NaN,07331,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,CHARCUTERIE MOULIN,NaN,NaN,NaN,47.22Z,NAFRev2,O
17416,850026550,00028,85002655000028,O,2021-04-28,NaN,NaN,4722ZB,2021-09-07T09:06:37,false,A,O,NaN,2019-04-01,5710,L'ENTRE-POT'S,NaN,NaN,NaN,NaN

In [34]:
donnees_sirene_fr["departement"]=donnees_sirene_fr.codeCommuneEtablissement.str[:2]

import numpy as np
donnees_sirene_fr["departement"]=np.where(donnees_sirene_fr.codeCommuneEtablissement.str[:2]=='97', donnees_sirene_fr.codeCommuneEtablissement.str[:3],donnees_sirene_fr.codeCommuneEtablissement.str[:2])
donnees_sirene_fr_agregees_par_departement = donnees_sirene_fr.groupby("departement").size().reset_index(name='NbBoucheriesParDep').set_index("departement")
donnees_sirene_fr
donnees_sirene_fr_agregees_par_departement
donnees_sirene_fr_agregees_par_departement.describe()

,NbBoucheriesParDep
count,104.000000
mean,193.730769
std,168.350145
min,3.000000
25%,95.750000
50%,140.000000
75%,228.500000
max,877.000000


In [6]:
# donnees_sirene_fr_agregees_par_departement = donnees_sirene_fr.groupby().sum()

## 3- Téléchargement des données du ministère de l'intérieur

In [63]:
#Téléchargement des données de crimes et délits rapportés déclarés auprès des commissariats 
url_datagouv_min_interieur="https://www.data.gouv.fr/fr/datasets/r/d792092f-b1f7-4180-a367-d043200c1520"

donnees_police_nationale_2018=pd.read_excel(url_datagouv_min_interieur, sheet_name='Services PN 2018', skiprows=[1,2])
donnees_gendarmerie_nationale_2018=pd.read_excel(url_datagouv_min_interieur, sheet_name='Services GN 2018', skiprows=[1,2])

del donnees_police_nationale_2018[donnees_police_nationale_2018.columns[0]]
del donnees_gendarmerie_nationale_2018[donnees_gendarmerie_nationale_2018.columns[0]]
donnees_police_nationale_2018.head(10)
donnees_gendarmerie_nationale_2018.head(10)


,Départements,01,01.1,01.2,01.3,02,02.1,02.2,02.3,02.4,03,03.1,03.2,04,04.1,04.2,04.3,05,05.1,06,06.1,06.2,06.3,06.4,07,07.1,07.2,08,08.1,08.2,08.3,08.4,09,09.1,09.2,10,10.1,10.2,11,11.1,11.2,12,12.1,12.2,13,13.1,13.2,13.3,13.4,14,14.1,14.2,14.3,14.4,14.5,15,15.1,15.2,16,16.1,16.2,17,17.1,17.2,17.3,17.4,18,18.1,18.2,19,19.1,21,21.1,21.2,21.3,22,22.1,22.2,22.3,23,23.1,24,24.1,24.2,24.3,25,25.1,25.2,26,26.1,26.2,26.3,27,27.1,27.2,27.3,27.4,28,28.1,28.2,28.3,29,29.1,29.2,29.3,29.4,29.5,2A,2A.1,2A.2,2B,2B.1,2B.2,2B.3,30,30.1,30.2,30.3,30.4,31,31.1,31.2,31.3,31.4,32,32.1,33,33.1,33.2,33.3,33.4,33.5,33.6,34,34.1,34.2,34.3,34.4,35,35.1,35.2,35.3,35.4,36,36.1,36.2,37,37.1,37.2,37.3,38,38.1,38.2,38.3,38.4,38.5,38.6,39,39.1,39.2,40,40.1,40.2,41,41.1,41.2,42,42.1,42.2,43,43.1,43.2,44,44.1,44.2,44.3,44.4,44.5,45,45.1,45.2,45.3,46,46.1,46.2,47,47.1,47.2,47.3,48,48.1,49,49.1,49.2,49.3,50,50.1,50.2,50.3,51,51.1,51.2,51.3,52,52.1,52.2,53,53.1,54,54.1,54.2,54.3,55,55.1,56,56.1,56.2,56.3,57,57.1,57.2,57.3,57.4,57.5,58,58.1,58.2,59,59.1,59.2,59.3,59.4,59.5,59.6,60,60.1,60.2,60.3,60.4,60.5,61,61.1,61.2,62,62.1,62.2,62.3,62.4,62.5,63,63.1,63.2,63.3,63.4,63.5,64,64.1,64.2,64.3,64.4,65,65.1,65.2,66,66.1,66.2,66.3,67,67.1,67.2,67.3,67.4,67.5,68,68.1,68.2,68.3,69,69.1,69.2,69.3,69.4,70,70.1,71,71.1,71.2,71.3,71.4,72,72.1,72.2,73,73.1,73.2,74,74.1,74.2,74.3,74.4,76,76.1,76.2,76.3,76.4,76.5,77,77.1,77.2,77.3,77.4,78,78.1,78.2,79,79.1,79.2,80,80.1,80.2,80.3,81,81.1,81.2,82,82.1,83,83.1,83.2,83.3,83.4,84,84.1,84.2,84.3,85,85.1,85.2,86,86.1,86.2,87,87.1,87.2,88,88.1,88.2,89,89.1,89.2,91,91.1,91.2,95,95.1,95.2,971,971.1,971.2,972,972.1,972.2,973,973.1,973.2,974,974.1,974.2,978,987,987.1,988,988.1,988.2,988.3
0,Homicides pour voler et à l'occasion de vols,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Homicides pour d'autres motifs,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,1.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,2.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,2.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,2.0,1.0,0.0,2.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,2.0,0.0,0.0,1.0,0.0,0.0,1.0,2.0,3.0,1.0,2.0,3.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0

In [68]:
donnees_police_nationale_2018_par_dep=donnees_police_nationale_2018.T

donnees_police_nationale_2018_par_dep
donnees_police_nationale_2018_par_dep.columns = donnees_police_nationale_2018_par_dep.iloc[0]
donnees_police_nationale_2018_par_dep=donnees_police_nationale_2018_par_dep.iloc[1: , :]

donnees_police_nationale_2018_par_dep.drop(donnees_police_nationale_2018_par_dep[donnees_police_nationale_2018_par_dep.index.str[:2]=='98'].index,inplace=True)
donnees_police_nationale_2018_par_dep["departement"]=np.where(donnees_police_nationale_2018_par_dep.index.str[:2]=='97', donnees_police_nationale_2018_par_dep.index.str[:3],donnees_police_nationale_2018_par_dep.index.str[:2])

donnees_police_nationale_2018_par_dep = donnees_police_nationale_2018_par_dep.groupby("departement").sum() 
donnees_police_nationale_2018_par_dep

Départements,Règlements de compte entre malfaireurs,Homicides pour voler et à l'occasion de vols,Homicides pour d'autres motifs,Tentatives d'homicides pour voler et à l'occasion de vols,Tentatives homicides pour d'autres motifs,Coups et blessures volontaires suivis de mort,Autres coups et blessures volontaires criminels ou correctionnels,Prises d'otages à l'occasion de vols,Prises d'otages dans un autre but,Sequestrations,Menaces ou chantages pour extorsion de fonds,Menaces ou chantages dans un autre but,Atteintes à la dignité et à la personnalité,Violations de domicile,Vols à main armée contre des établissements financiers,Vols à main armée contre des éts industriels ou commerciaux,Vols à main armée contre des entreprises de transports de fonds,Vols à main armée contre des particuliers à leur domicile,Autres vols à main armée,"Vols avec armes blanches contre des établissements financiers,commerciaux ou industriels",Vols avec armes blanches contre des particuliers à leur domicile,Autres vols avec armes blanches,"Vols violents sans arme contre des établissements financiers,commerciaux ou industriels",Vols violents sans arme contre des particuliers à leur domicile,Vols violents sans arme contre des femmes sur voie publique ou autre lieu public,Vols violents sans arme contre d'autres victimes,Cambriolages de locaux d'habitations principales,Cambriolages de résidences secondaires,"Camb.de locaux industriels, commerciaux ou financiers",Cambriolages d'autres lieux,Vols avec entrée par ruse en tous lieux,Vols à la tire,Vols à l'étalage,Vols de véhicules de transport avec frêt,Vols d'automobiles,Vols de véhicules motorisés à 2 roues,Vols à la roulotte,Vols d''accessoires sur véhicules à moteur immatriculés,Vols simples sur chantier,Vols simples sur exploitations agricoles,Autres vols simples contre des établissements publics ou privés,Autres vols simples contre des particuliers dans deslocaux privés,Autres vols simples contre des particuliers dans des locaux ou lieux publics,Recels,Proxénétisme,Viols sur des majeur(e)s,Viols sur des mineur(e)s,Harcèlements sexuels et autres agressions sexuelles contre des majeur(e)s,Harcèlements sexuels et autres agressions sexuelles contre des mineur(e)s,Atteintes sexuelles,Homicides commis contre enfants de moins de 15 ans,"Violences, mauvais traitements et abandons d'enfants.",Délits au sujet de la garde des mineurs,Non versement de pension alimentaire,Trafic et revente sans usage de stupéfiants,Usage-revente de stupéfiants,Usage de stupéfiants,Autres infractions à la législation sur les stupéfiants,Délits de débits de boissons et infraction à la règlementation sur l'alcool et le tabac,Fraudes alimentaires et infractions à l'hygiène,Autres délits contre santé publique et la réglementation des professions médicales,Incendies volontaires de biens publics,Incendies volontaires de biens privés,Attentats à l'explosif contre des biens publics,Attentats à l'explosif contre des biens privés,Autres destructions er dégradations de biens publics,Autres destructions er dégradations de biens privés,Destructions et dégradations de véhicules privés,Infractions aux conditions générales d'entrée et de séjour des étrangers,"Aide à l'entrée, à la circulation et au séjour des étrangers",Autres infractions à la police des étrangers,Outrages à dépositaires autorité,Violences à dépositaires autorité,Port ou détention armes prohibées,Atteintes aux intérêts fondamentaux de la Nation,Délits des courses et des jeux,Délits interdiction de séjour et de paraître,"Destructions, cruautés et autres délits envers les animaux",Atteintes à l'environnement,Chasse et pêche,Faux documents d'identité,Faux documents concernant la circulation des véhicules,Autres faux documents administratifs,Faux en écriture publique et authentique,Autres faux en écriture,Fausse monnaie,Contrefaçons et fraudes industrielles et commerciales,Contrefaçons littéraires et artistique,Falsification et usages de chèques volés,Falsification et usages de cartes de 

* Agrégation par département, transposition et fusion :

In [77]:
donnees_police_nationale_2018_par_dep = donnees_police_nationale_2018_par_dep.sum(axis=1).reset_index("departement").set_index("departement")
donnees_police_nationale_2018_par_dep.rename(columns={0: "Total des crimes et délits de la Police Nationale"})

,Total des crimes et délits de la Police Nationale
departement,
01,5877
02,12147
03,7953
04,2759
05,2919
...,...
971,9962
972,8385
973,7025


In [80]:
donnees_gendarmerie_nationale_2018_par_dep=donnees_gendarmerie_nationale_2018.T

donnees_gendarmerie_nationale_2018_par_dep
donnees_gendarmerie_nationale_2018_par_dep.columns = donnees_gendarmerie_nationale_2018_par_dep.iloc[0]
donnees_gendarmerie_nationale_2018_par_dep=donnees_gendarmerie_nationale_2018_par_dep.iloc[1: , :]

donnees_gendarmerie_nationale_2018_par_dep.drop(donnees_gendarmerie_nationale_2018_par_dep[donnees_gendarmerie_nationale_2018_par_dep.index.str[:2]=='98'].index,inplace=True)
donnees_gendarmerie_nationale_2018_par_dep["departement"]=np.where(donnees_gendarmerie_nationale_2018_par_dep.index.str[:2]=='97', donnees_gendarmerie_nationale_2018_par_dep.index.str[:3],donnees_gendarmerie_nationale_2018_par_dep.index.str[:2])

donnees_gendarmerie_nationale_2018_par_dep = donnees_gendarmerie_nationale_2018_par_dep.groupby("departement").sum() 
donnees_gendarmerie_nationale_2018_par_dep

Départements,Homicides pour voler et à l'occasion de vols,Homicides pour d'autres motifs,Tentatives d'homicides pour voler et à l'occasion de vols,Tentatives homicides pour d'autres motifs,Coups et blessures volontaires suivis de mort,Autres coups et blessures volontaires criminels ou correctionnels,Prises d'otages à l'occasion de vols,Prises d'otages dans un autre but,Sequestrations,Menaces ou chantages pour extorsion de fonds,Menaces ou chantages dans un autre but,Atteintes à la dignité et à la personnalité,Violations de domicile,Vols à main armée contre des établissements financiers,Vols à main armée contre des éts industriels ou commerciaux,Vols à main armée contre des entreprises de transports de fonds,Vols à main armée contre des particuliers à leur domicile,Autres vols à main armée,"Vols avec armes blanches contre des établissements financiers,commerciaux ou industriels",Vols avec armes blanches contre des particuliers à leur domicile,Autres vols avec armes blanches,"Vols violents sans arme contre des établissements financiers,commerciaux ou industriels",Vols violents sans arme contre des particuliers à leur domicile,Vols violents sans arme contre des femmes sur voie publique ou autre lieu public,Vols violents sans arme contre d'autres victimes,Cambriolages de locaux d'habitations principales,Cambriolages de résidences secondaires,"Camb.de locaux industriels, commerciaux ou financiers",Cambriolages d'autres lieux,Vols avec entrée par ruse en tous lieux,Vols à la tire,Vols à l'étalage,Vols de véhicules de transport avec frêt,Vols d'automobiles,Vols de véhicules motorisés à 2 roues,Vols à la roulotte,Vols d''accessoires sur véhicules à moteur immatriculés,Vols simples sur chantier,Vols simples sur exploitations agricoles,Autres vols simples contre des établissements publics ou privés,Autres vols simples contre des particuliers dans deslocaux privés,Autres vols simples contre des particuliers dans des locaux ou lieux publics,Recels,Proxénétisme,Viols sur des majeur(e)s,Viols sur des mineur(e)s,Harcèlements sexuels et autres agressions sexuelles contre des majeur(e)s,Harcèlements sexuels et autres agressions sexuelles contre des mineur(e)s,Atteintes sexuelles,Homicides commis contre enfants de moins de 15 ans,"Violences, mauvais traitements et abandons d'enfants.",Délits au sujet de la garde des mineurs,Non versement de pension alimentaire,Trafic et revente sans usage de stupéfiants,Usage-revente de stupéfiants,Usage de stupéfiants,Autres infractions à la législation sur les stupéfiants,Délits de débits de boissons et infraction à la règlementation sur l'alcool et le tabac,Fraudes alimentaires et infractions à l'hygiène,Autres délits contre santé publique et la réglementation des professions médicales,Incendies volontaires de biens publics,Incendies volontaires de biens privés,Attentats à l'explosif contre des biens publics,Attentats à l'explosif contre des biens privés,Autres destructions er dégradations de biens publics,Autres destructions er dégradations de biens privés,Destructions et dégradations de véhicules privés,Infractions aux conditions générales d'entrée et de séjour des étrangers,"Aide à l'entrée, à la circulation et au séjour des étrangers",Autres infractions à la police des étrangers,Outrages à dépositaires autorité,Violences à dépositaires autorité,Port ou détention armes prohibées,Atteintes aux intérêts fondamentaux de la Nation,Délits des courses et des jeux,Délits interdiction de séjour et de paraître,"Destructions, cruautés et autres délits envers les animaux",Atteintes à l'environnement,Chasse et pêche,Faux documents d'identité,Faux documents concernant la circulation des véhicules,Autres faux documents administratifs,Faux en écriture publique et authentique,Autres faux en écriture,Fausse monnaie,Contrefaçons et fraudes industrielles et commerciales,Contrefaçons littéraires et artistique,Falsification et usages de chèques volés,Falsification et usages de cartes de crédit,Escroqueries et abus de confianc

In [83]:
donnees_gendarmerie_nationale_2018_par_dep = donnees_gendarmerie_nationale_2018_par_dep.sum(axis=1).reset_index("departement").set_index("departement")
donnees_gendarmerie_nationale_2018_par_dep.rename(columns={0: "Total des crimes et délits de la Gendarmerie Nationale"})

,Total des crimes et délits de la Gendarmerie Nationale
departement,
01,20210.0
02,13695.0
03,5463.0
04,5608.0
05,3672.0
...,...
971,13493.0
972,9447.0
973,10754.0


In [91]:
merged_crimes_delits_par_dep = pd.merge(donnees_police_nationale_2018_par_dep, donnees_gendarmerie_nationale_2018_par_dep, on="departement")
merged_crimes_delits_par_dep.rename(columns={0: "Total des crimes et délits de la Police Nationale" , 1: "Total des crimes et délits de la Gendarmerie Nationale"})


,0_x,0_y
departement,,
01,5877,20210.0
02,12147,13695.0
03,7953,5463.0
04,2759,5608.0
05,2919,3672.0
...,...,...
95,66605,14018.0
971,9962,13493.0
972,8385,9447.0


## 4 - Téléchargement des revenus localisés

In [10]:
#Téléchargement des données Filosofi au niveau IRIS (millénisme 2018)
URL_FILO_DECL_2018="https://www.insee.fr/fr/statistiques/fichier/5055909/BASE_TD_FILO_DEC_IRIS_2018.xlsx"
URL_FILO_DISP_2018="https://www.insee.fr/fr/statistiques/fichier/5055909/BASE_TD_FILO_DISP_IRIS_2018.xlsx"

donnees_filo_decl_2018=pd.read_excel(URL_FILO_DECL_2018, sheet_name='IRIS_DEC', skiprows=5)
donnees_filo_disp_2018=pd.read_excel(URL_FILO_DISP_2018, sheet_name='IRIS_DISP', skiprows=5)

In [11]:
print(donnees_filo_decl_2018.head(10))
print(donnees_filo_disp_2018.head(10))

        IRIS                           LIBIRIS    COM             LIBCOM  \
0  010040101  Les Perouses-Triangle d'Activité  01004  Ambérieu-en-Bugey   
1  010040102                     Longeray-Gare  01004  Ambérieu-en-Bugey   
2  010040201    Centre-Saint-Germain-Vareilles  01004  Ambérieu-en-Bugey   
3  010040202                 Tiret-Les Allymes  01004  Ambérieu-en-Bugey   
4  010330102                      Centre Ville  01033         Valserhône   
5  010330103             Lancrans-Coupy-Vanchy  01033         Valserhône   
6  010330201        Arc Vouvray-Gare-Châtillon  01033         Valserhône   
7  010330202               Plateau de Musinens  01033         Valserhône   
8  010330301                             Arlod  01033         Valserhône   
9  010330401            Châtillon-en-Michaille  01033         Valserhône   

   DEC_PIMP18  DEC_TP6018  DEC_Q118  DEC_MED18  DEC_Q318  DEC_EQ18  ...  \
0        45.4        31.0   11150.0    18540.0   25900.0      0.80  ...   
1        39.3

In [12]:
URL_FILO_ZIP_2018="https://www.insee.fr/fr/statistiques/fichier/5009236/base-cc-filosofi-2018_CSV_geo2021.zip"
#Télécharger le zip de l'URL
r=requests.get(URL_FILO_ZIP_2018)

open("base-cc-filosofi-2018_CSV_geo2021.zip", 'wb').write(r.content)
    
    
with ZipFile("base-cc-filosofi-2018_CSV_geo2021.zip",'r') as myzip:
    data = myzip.open("cc_filosofi_2018_COM-geo2021.CSV")
    data2 = myzip.open("cc_filosofi_2018_DEP-geo2021.CSV")
    
donnees_filo_communes_2018=pd.read_csv(data,sep=';')    
donnees_filo_departement_2018=pd.read_csv(data2,sep=';')    

/opt/conda/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [13]:
donnees_filo_communes_2018.head(10)
donnees_filo_departement_2018.head(10)

,CODGEO,NBMENFISC18,NBPERSMENFISC18,MED18,PIMP18,TP6018,TP60AGE118,TP60AGE218,TP60AGE318,TP60AGE418,...,PPEN18,PPAT18,PPSOC18,PPFAM18,PPMINI18,PPLOGT18,PIMPOT18,D118,D918,RD18
0,01,258018,618263,23420,53.6,10.3,14.9,11.8,11.4,10.1,...,24.9,9.6,4.0,1.9,1.2,0.9,-17.3,12900,44080,3.4
1,02,221370,512838,19690,43.6,18.4,29.7,22.8,21.2,18.4,...,30.6,8.3,7.9,2.9,3.2,1.8,-15.2,10730,33170,3.1
2,03,157528,325813,20360,44.0,15.5,25.3,19.7,18.2,16.1,...,37.6,10.5,6.6,2.0,3.0,1.6,-15.5,11360,33600,3.0
3,04,73620,154535,20580,45.6,16.8,24.9,20.9,20.3,17.2,...,35.7,14.6,5.7,1.9,2.4,1.4,-16.6,11060,35380,3.2
4,05,63474,133391,20880,47.5,13.9,19.9,16.9,16.6,14.6,...,35.5,11.4,5.3,1.9,2.1,1.3,-16.2,11800,35410,3.0
5,06,520415,1099477,22140,55.2,15.8,21.7,17.6,17.6,16.0,...,29.7,13.5,4.8,1.5,2.1,1.2,-20.6,11090,41770,3.8
6,07,141763,311917,20780,45.0,14.4,21.5,18.3,17.5,14.1,...,34.0,12.2,5.5,2.1,2.1,1.3,-15.9,11630,34440,3.0
7,08,117199,261731,19650,41.5,18.9,29.7,22.6,22.1,18.2,...,31.3,9.2,7.8,2.6,3.3,1.9,-14.8,10660,32960,3.1
8,09,68614,143949,19820,40.7,18.0,26.7,22.8,21.0,18.7,...,37.4,9.6,7.0,2.0,3.3,1.7,-14.8,10690,33090,3.1
9,10,130958,288559,20430,46.6,16.2,27.6,21.1,18.6,15.1,...,30.8,10.1,6.8,2.4,2.7,1.7,-16.5,11240,35650,3.2


## 5 - Téléchargement des populations légales

In [14]:
#Téléchargement des populations légales du millénisme 2018 
URL_POP_LEG="https://www.insee.fr/fr/statistiques/fichier/4989724/ensemble.zip"

#Télécharger le zip de l'URL
r=requests.get(URL_POP_LEG)

open("ensemble.zip", 'wb').write(r.content)
    
    
with ZipFile("ensemble.zip",'r') as myzip:
    data = myzip.open("Departements.csv")

donnees_pop_leg_18=pd.read_csv(data,sep=';')     

In [15]:
donnees_pop_leg_18.sample(10)

,CODREG,REG,CODDEP,DEP,NBARR,NBCAN,NBCOM,PMUN,PTOT
57,44,Grand Est,57,Moselle,5,27.0,725,1043524,1062362
32,76,Occitanie,32,Gers,3,17.0,461,191283,198043
97,2,Martinique,972,Martinique,4,NaN,34,368783,373762
2,84,Auvergne-Rhône-Alpes,03,Allier,3,19.0,317,337171,346156
74,84,Auvergne-Rhône-Alpes,74,Haute-Savoie,4,17.0,279,816699,838044
98,3,Guyane,973,Guyane,2,NaN,22,276128,278472
23,75,Nouvelle-Aquitaine,23,Creuse,2,15.0,256,117503,121009
31,76,Occitanie,31,Haute-Garonne,3,27.0,586,1380672,1403685
24,75,Nouvelle-Aquitaine,24,Dordogne,4,25.0,505,413418,423868
86,75,Nouvelle-Aquitaine,86,Vienne,3,19.0,266,437586,447895


## 6 - Téléchargement des données d'emplois localisées

In [16]:
#Téléchargement des données d'emploi localisées
donnees_emploi_statut_18 = pynsee.download.telechargerDonnees("ESTEL_T201", date = "2018")
donnees_emploi_statut_18.sample(10)

donnees_emploi_sexe_18 = pynsee.download.telechargerDonnees("ESTEL_T202", date = "2018")
donnees_emploi_sexe_18.sample(10)


URL_TAUX_CHOMAGE_PAR_DEP="https://www.insee.fr/fr/statistiques/fichier/2012804/sl_etc_2021T2.xls"


donnees_chomage_par_departement_series_longues=pd.read_excel(URL_TAUX_CHOMAGE_PAR_DEP, sheet_name='Département', skiprows=3)

Data will be stored in the following location: /tmp/tmpkomfvkcv


Downloading: 100%|██████████| 2.26M/2.26M [00:02<00:00, 1.01MiB/s]


No destination directory defined. Data have been written there: /tmp/tmpthd1cj7o


Extracting: 100%|██████████| 13.6M/13.6M [00:00<00:00, 185MB/s]


Data will be stored in the following location: /tmp/tmpnl9bqbbr


Downloading: 100%|██████████| 2.26M/2.26M [00:02<00:00, 1.14MiB/s]


No destination directory defined. Data have been written there: /tmp/tmpqigq592_


Extracting: 100%|██████████| 13.6M/13.6M [00:00<00:00, 181MB/s]
/opt/conda/lib/python3.9/site-packages/pynsee/download/__init__.py:73: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  return chargerDonnees(


In [17]:
donnees_chomage_par_departement_series_longues.head(10)

,Code,Libellé,T1_1982,T2_1982,T3_1982,T4_1982,T1_1983,T2_1983,T3_1983,T4_1983,...,T1_2019,T2_2019,T3_2019,T4_2019,T1_2020,T2_2020,T3_2020,T4_2020,T1_2021,T2_2021
0,01,AIN,3.8,3.9,4.0,4.1,4.2,4.2,4.4,4.6,...,6.2,6.0,6.2,5.9,5.7,5.6,7.0,6.0,6.1,5.9
1,02,AISNE,8.3,8.5,8.6,8.5,8.5,8.6,8.8,9.2,...,12.1,11.8,11.9,11.4,11.2,9.7,12.6,11.0,11.2,11.0
2,03,ALLIER,7.4,7.6,7.8,7.8,7.8,7.8,7.9,8.5,...,9.3,9.1,9.1,8.7,8.5,8.0,9.8,8.3,8.4,8.5
3,04,ALPES-DE-HAUTE-PROVENCE,5.6,5.7,5.9,6.1,6.2,6.2,6.4,6.7,...,10.5,10.2,10.2,9.5,9.2,8.8,10.6,9.0,9.4,9.5
4,05,HAUTES-ALPES,4.9,5.0,5.1,5.4,5.8,5.9,6.1,6.5,...,8.2,7.9,7.9,7.6,7.3,6.9,8.4,7.5,9.2,7.8
5,06,ALPES-MARITIMES,8.5,8.8,9.2,9.2,9.2,9.4,9.5,9.8,...,9.2,8.8,8.8,8.4,8.1,8.1,10.2,8.9,8.9,9.0
6,07,ARDECHE,6.5,6.7,7.0,7.1,7.1,7.2,7.5,7.7,...,9.4,9.1,9.2,8.8,8.5,8.1,9.9,8.4,8.7,9.0
7,08,ARDENNES,8.7,8.9,9.0,9.0,8.9,8.9,9.2,9.7,...,10.0,9.9,10.3,9.9,9.6,8.5,11.0,9.4,9.3,9.5
8,09,ARIEGE,7.5,7.5,7.4,7.4,7.4,7.2,6.9,7.0,...,11.1,10.6,10.8,10.3,9.9,8.7,11.2,9.8,10.0,9.7
9,10,AUBE,6.2,6.1,6.0,6.0,5.8,5.8,6.2,6.5,...,11.2,10.6,10.7,10.3,10.1,8.7,11.0,10.0,10.2,9.9


## 7 - Téléchargement du Code Officiel Géographique

* Pour des raisons de simplicité, nous travaillons en géographie 2021 :

In [18]:
#cog_commune = pynsee.download.telechargerDonnees("COG_COMMUNE", date = "dernier")
URL_COG_2021="https://www.insee.fr/fr/statistiques/fichier/5057840/commune2021-csv.zip"

#Télécharger le zip de l'URL
r=requests.get(URL_COG_2021)

open("commune2021-csv.zip", 'wb').write(r.content)
    
    
with ZipFile("commune2021-csv.zip",'r') as myzip:
    data = myzip.open("commune2021.csv")

donnees_cog_2021=pd.read_csv(data)     

In [19]:
donnees_cog_2021

,TYPECOM,COM,REG,DEP,CTCD,ARR,TNCC,NCC,NCCENR,LIBELLE,CAN,COMPARENT
0,COM,01001,84.0,01,01D,012,5,ABERGEMENT CLEMENCIAT,Abergement-Clémenciat,L'Abergement-Clémenciat,0108,NaN
1,COM,01002,84.0,01,01D,011,5,ABERGEMENT DE VAREY,Abergement-de-Varey,L'Abergement-de-Varey,0101,NaN
2,COM,01004,84.0,01,01D,011,1,AMBERIEU EN BUGEY,Ambérieu-en-Bugey,Ambérieu-en-Bugey,0101,NaN
3,COM,01005,84.0,01,01D,012,1,AMBERIEUX EN DOMBES,Ambérieux-en-Dombes,Ambérieux-en-Dombes,0122,NaN
4,COM,01006,84.0,01,01D,011,1,AMBLEON,Ambléon,Ambléon,0104,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
37737,COM,97613,6.0,976,976D,NaN,0,M TSANGAMOUJI,M'Tsangamouji,M'Tsangamouji,97613,NaN
37738,COM,97614,6.0,976,976D,NaN,1,OUANGANI,Ouangani,Ouangani,97610,NaN
37739,COM,97615,6.0,976,976D,NaN,0,PAMANDZI,Pamandzi,Pamandzi,97611,NaN
37740,COM,97616,6.0,976,976D,NaN,0,SADA,Sada,Sada,97612,NaN


## 8 - Fusion des données

In [20]:
# donnees_2018=join()